# Prepare and understand data for modeling
모든 데이터는 보통 지저분하고 데이터가 의도한 것에 대한 충분한 신뢰성을 가지고 있지 않다.  

데이터는 중복 데이터나 관찰되지않는 값(결측치), 아웃라이어(이상치), 존재하지 않는 주소, 잘못된 전화번호 또는 지역코드, 올바르지 않은 직역좌표, 잘못된 테이터나 레이블, 대소문자 구분, 공백 관련 문제를 가지고있다.  

데이터 과학자, 데이터 엔지니어는 통계모델 또는 머신러닝 모델을 빌드하기 위해 이러한 데이터를 깨끗하게 만들어야 한다.  

데이터는 앞에서 말한 문제점들이 없을 경우 기술적으로 깨끗하다고 말 할 수 있다. 그러나 모델링 목적으로 데이터셋을 깨끗하게 하기 위해서는 피처의 분포를 확인해야 하고 사전에 정의된 조건들을 만족하는지 검증 해야 한다.

**데이터과학자는 80 - 90%의 시간을 데이터를 다루거나 피처에 익숙해지는데 쓰게 된다**

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('appName1').setMaster('local[2]')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# Duplicates

In [2]:
df = spark.createDataFrame([
    (1, 144.5, 5.9, 33, 'M'),
    (2, 167.2, 5.4, 45, 'M'),
    (3, 124.1, 5.2, 23, 'F'),
    (4, 144.5, 5.9, 33, 'M'),
    (5, 133.2, 5.7, 54, 'F'),
    (3, 124.1, 5.2, 23, 'F'),
    (5, 129.2, 5.3, 42, 'M')
], ['id', 'weight', 'height', 'age', 'gender'])

In [3]:
print('count of rows: {0}'.format(df.distinct().count()))

count of rows: 6


In [4]:
df = df.dropDuplicates()
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [5]:
print('count of rows: {0}'.format(df.count()))

count of rows: 6


In [6]:
print('count of distinct ids: {0}'\
      .format(df.select([c for c in df.columns if c != 'id']).distinct().count()))

count of distinct ids: 5


In [7]:
df.columns

['id', 'weight', 'height', 'age', 'gender']

In [8]:
[c for c in df.columns if c != 'id']

['weight', 'height', 'age', 'gender']

In [10]:
df = df.dropDuplicates(subset=[c for c in df.columns if c != 'id'])
# df = df.dropDuplicates(subset=['weight', 'height', 'age', 'gender'])
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [2]:
import pyspark.sql.functions as fn

In [12]:
df.agg(
        fn.count('id').alias('count'),
        fn.countDistinct('id').alias('distinct')
).show()

+-----+--------+
|count|distinct|
+-----+--------+
|    5|       4|
+-----+--------+



monotonically_increasing_id() : 각 행에 고유한 값을 부여 한다.  
각각의 파티션에 800억개의 데이터가 있고 파티션의 수가 약 10억개 미만인 데이터들에 대해서는 안정적으로 공유한 ID값을 부여 해 준다. (스파크 이전 버전에서는 같은 데이터프레임에서 여러번 작업이 이뤄졌을때마다 ID값이 바뀌었다, 스파크 2.0에서는 고정된 ID값을 부여 해 준다.)

In [13]:
df.withColumn('new_id', fn.monotonically_increasing_id()).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  5| 133.2|   5.7| 54|     F|  25769803776|
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  2| 167.2|   5.4| 45|     M| 592705486848|
|  3| 124.1|   5.2| 23|     F|1236950581248|
|  5| 129.2|   5.3| 42|     M|1365799600128|
+---+------+------+---+------+-------------+



# Missing observations
**미관찰 값들을 다룰 수 있는 가장 간단한 방법은 미관찰 값을 가지고있는 모든 데이터를 제거 하는 것이다.**  
단 너무 많은 데이터를 제거하지 않도록 조심해야함.  
데이터셋 전체에서 미관찰 값의 분포에 따라 데이터셋 전체의 사용 가능성에 큰 영향을 미치리 수 있다.  
데이터를 제거한 후 아주 작은 데이터만 남거나, 데이터가 절반 이상으로 줄어들었다면 어떤 피처가 빈칸을 가장 많이 가지고있는지 확인하고 그 피처를 제거하는 것이 더 좋다.  

미관찰 값을 다루는 또 다른 방법은 None으로 채우는 것이다. 이럴경우 데이터필드 타입에 딸라 몇몇 다양한 값으로 대체해 채워 넣을 수 있다.
- 데이터가 참/거짓으로 구분되면 Missing라는 세번째 카테고리를 넣는다
- 데이터가 이미 카테고리를 가지고 있다면 Missing카테고리를 추가한다
- 순서 혹은 숫자 데이터를 가지고있을경우에는 평균, 중간값 또는 미리 정의된 다른 값으로 바꿀 수 있다.

In [14]:
df_miss = spark.createDataFrame([
    (1, 144.5, 5.9, 33, 'M', 100000),
    (2, 167.2, 5.4, 45, 'M', None),
    (3, None, 5.2, None, None, None),
    (4, 144.5, 5.9, 33, 'M', None),
    (5, 133.2, 5.7, 54, 'F', None),
    (6, 124.1, 5.2, None, 'F', None),
    (7, 129.2, 5.3, 42, 'M', 76000)
], ['id', 'weight', 'height', 'age', 'gender', 'income'])

In [15]:
# id별 결측치 갯수 확인
df_miss.rdd.map(lambda row: (row['id'], sum([c == None for c in row]))).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [16]:
df_miss.where('id ==  3').show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  3|  null|   5.2|null|  null|  null|
+---+------+------+----+------+------+



In [17]:
# 컬럼별 결측치 확인
df_miss.agg(*[
    (1 - (fn.count(c) / fn.count('*'))).alias(c + 'missing')
    for c in df_miss.columns
]).show()

+---------+------------------+-------------+------------------+------------------+------------------+
|idmissing|     weightmissing|heightmissing|        agemissing|     gendermissing|     incomemissing|
+---------+------------------+-------------+------------------+------------------+------------------+
|      0.0|0.1428571428571429|          0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+---------+------------------+-------------+------------------+------------------+------------------+



In [19]:
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c != 'income'])

In [21]:
[c for c in df_miss.columns if c != 'income']

['id', 'weight', 'height', 'age', 'gender']

In [20]:
df_miss_no_income.show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 144.5|   5.9|  33|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [22]:
df_miss.columns

['id', 'weight', 'height', 'age', 'gender', 'income']

In [23]:
df_miss.select(['id', 'weight', 'height', 'age', 'gender']).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 144.5|   5.9|  33|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [24]:
df_miss_no_income.dropna(thresh=3).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 144.5|   5.9|  33|     M|
|  2| 167.2|   5.4|  45|     M|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



미관찰 값을 추정해 채우려면 fillna()함수를 사용 할 수 있다.  
이 함수는 단일 integer, float, long, string타입을 지원한다.  
전체 데이터셋에서 미관찰값은 지정한 값으로 채워질 것이다.  
평균, 중앙값 또는 다른 계산된 값으로 채우려면 우선 그 값을 계산하고 그 값을 가지는 딕셔너리를 만든 후 fillna()함수에 전달 하면된다.

In [25]:
means = df_miss_no_income.agg(
    *[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c != 'gender']
).toPandas().to_dict('records')[0]

toPandas()함수는 collect()함수와같은 방식으로 동작한다, 데이터양에따라 문제가 발생할수있다. toPandas()함수는 모든 정보를 워커노트로부터 수집 후 드라이버 노드로 가지고 온다.

means

In [27]:
means['gender'] = 'missing'
means

{'id': 4.0,
 'weight': 140.45000000000002,
 'height': 5.514285714285714,
 'age': 41.4,
 'gender': 'missing'}

In [28]:
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             144.5|   5.9| 33|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.45000000000002|   5.2| 41|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 41|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



# Outliers

In [39]:
df_outliers = spark.createDataFrame([
    (1, 144.5, 5.9, 33),
    (2, 167.2, 5.4, 45),
    (3, 342.3, 5.2, 99),
    (4, 144.5, 5.9, 33),
    (5, 133.2, 5.7, 54),
    (6, 124.1, 5.2, 23),
    (7, 129.2, 5.3, 42)
], ['id', 'weight', 'height', 'age'])

In [40]:
df_outliers.printSchema()

root
 |-- id: long (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)
 |-- age: long (nullable = true)



아웃라이어는 대부분의 데이터와는 매우 다른 분포를 띠고있는 데이터를 말함.  
매우 다르다는 것에 대한 정의는 각자 다를 수 있다.  

**일반적으로 모든 값들이 대체적으로 Q1 - 1.5IQR ~ Q3 + 1.5IQR 사이에있는 데이터이면 아웃라이어가 없다고 말 할 수 있다.**

IQR은 상위 쿼타일(75%)와 하위 쿼타일(25%)의 차로 정의 된다.

approxQuantile()함수를 사용해서 이상치 값을 확인 할 수 있다.
- 첫번째 : 칼럼명
- 두번째 : 0과 1사이 값 (0.5 중간 값)
- 세번째 : 각 피처에 대한 허용 가능한 수준의 에러(0으로 지정시 피처에 대한 정확한 값을 계산 할 수 있으나 매우 많은 연산을 수행한다)

In [31]:
cols = ['weight', 'height', 'age']
bounds = {}
for col in cols:
    quantiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]

In [32]:
bounds

{'weight': [72.19999999999999, 224.2],
 'height': [4.15, 6.950000000000001],
 'age': [1.5, 85.5]}

In [53]:
outliers = df_outliers.select(
    *['id'] + 
    [(
        (df_outliers[c] < bounds[c][0]) |
        (df_outliers[c] > bounds[c][1])
     ).alias(c + '_o') for c in cols ]
)

In [54]:
outliers.show()

+---+--------+--------+-----+
| id|weight_o|height_o|age_o|
+---+--------+--------+-----+
|  1|   false|   false|false|
|  2|   false|   false|false|
|  3|    true|   false| true|
|  4|   false|   false|false|
|  5|   false|   false|false|
|  6|   false|   false|false|
|  7|   false|   false|false|
+---+--------+--------+-----+



In [ ]:
df_outliers = df_outliers.join(outliers, on='id')
df_outliers.show()

In [ ]:
df_outliers.filter('weight_o').select('id', 'weight').show()

In [ ]:
df_outliers.filter('age_o').select('id', 'age').show()

# Understand data
기술 통계 : 기술 통계는 데이터셋에서의 관찰 값 갯수, 각 컬럼의 평균, 표준 편차 또는 최댓값, 최솟값 등의 기본 적인 정보를 확인 한다.

In [3]:
import pyspark.sql.types as typ

In [4]:
fraud = sc.textFile('./ccFraud.csv.gz')

In [5]:
header = fraud.first()
header

'"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"'

In [6]:
fraud = fraud.filter(lambda row: row != header) \
             .map(lambda row: [int(elem) for elem in row.split(',')])

In [7]:
fields = [
    *[
        typ.StructField(h[1:-1], typ.IntegerType(), True) for h in header.split(',')
    ]
]

In [8]:
fields

[StructField(custID,IntegerType,true),
 StructField(gender,IntegerType,true),
 StructField(state,IntegerType,true),
 StructField(cardholder,IntegerType,true),
 StructField(balance,IntegerType,true),
 StructField(numTrans,IntegerType,true),
 StructField(numIntlTrans,IntegerType,true),
 StructField(creditLine,IntegerType,true),
 StructField(fraudRisk,IntegerType,true)]

In [9]:
schema = typ.StructType(fields)

In [10]:
fraud_df = spark.createDataFrame(fraud, schema)

In [11]:
fraud_df.printSchema()

root
 |-- custID: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- state: integer (nullable = true)
 |-- cardholder: integer (nullable = true)
 |-- balance: integer (nullable = true)
 |-- numTrans: integer (nullable = true)
 |-- numIntlTrans: integer (nullable = true)
 |-- creditLine: integer (nullable = true)
 |-- fraudRisk: integer (nullable = true)



In [67]:
# 빈도수 구하기
fraud_df.groupby('gender').count().show()

+------+-------+
|gender|  count|
+------+-------+
|     1|6178231|
|     2|3821769|
+------+-------+



In [12]:
# descrive() 확인
numerical = ['balance', 'numTrans', 'numIntlTrans']

In [70]:
desc = fraud_df.describe(numerical)

In [71]:
desc.show()

+-------+-----------------+------------------+-----------------+
|summary|          balance|          numTrans|     numIntlTrans|
+-------+-----------------+------------------+-----------------+
|  count|         10000000|          10000000|         10000000|
|   mean|     4109.9199193|        28.9351871|        4.0471899|
| stddev|3996.847309737077|26.553781024522852|8.602970115863767|
|    min|                0|                 0|                0|
|    max|            41485|               100|               60|
+-------+-----------------+------------------+-----------------+



기술 통계는 적은 정보이지만 많은 것들을 알 수 있다.  
- 모든 피처는 양의 방향으로 왜곡이 돼었다 (최대값이 평균보다 몇배 더 크다)
- 변동계수 COEFFICIENT VARIABIOIN 가 매우 크다 (값이 1과 가깝거나 크다, 이는 넓게 퍼진 데이터를 의미)

In [72]:
# 비대칭 확인
fraud_df.agg({'balance': 'skewness'}).show()

+------------------+
| skewness(balance)|
+------------------+
|1.1818315552995033|
+------------------+



In [73]:
# Correlations
# corr() 함수는 두쌍의 상관계수만 계산 할 수 있다, 피어슨 상관계수를 지원한다.
fraud_df.corr('balance', 'numTrans')

0.00044523140172659576

In [76]:
n_numerical = len(numerical)
corr = []
for i in range(0, n_numerical):
    temp = [None] * i
    for j in range(i, n_numerical):
        temp.append(fraud_df.corr(numerical[i], numerical[j]))
    corr.append(temp)

In [78]:
# 'balance','numTrans', 'numIntlTrans'
corr

[[1.0, 0.00044523140172659576, 0.00027139913398184604],
 [None, 1.0, -0.0002805712819816179],
 [None, None, 1.0]]

# Visualization

In [77]:
!conda list holoviews

# packages in environment at C:\Users\a\Anaconda3:
#
# Name                    Version                   Build  Channel


In [79]:
!conda list bkcharts

# packages in environment at C:\Users\a\Anaconda3:
#
# Name                    Version                   Build  Channel
bkcharts                  0.2                      py37_0  


In [80]:
!conda install -c conda-forge holoviews -y

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\a\Anaconda3

  added / updated specs:
    - holoviews


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    holoviews-1.12.7           |     pyh9f0ad1d_1         3.2 MB  conda-forge
    matplotlib-base-3.2.1      |   py37h911224e_0         7.1 MB  conda-forge
    param-1.9.3                |             py_0          60 KB  conda-forge
    pyviz_comms-0.7.6          |     pyh9f0ad1d_0          13 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        10.5 MB

The following NEW packages will be INSTALL

### Histograms
히스토그램은 피처의 분포를 시각화하는 가장 쉬운 방법이다.  
스파크에서 히스토그램을 만드는 방법은 3가지이다.
- 데이터를 워커노드에 집계해서 워커노드가 bin리스트를 드라이버 노드에게 리턴하고 각 bin의 개수를 드라이버 노드가 카운트
- 데이터를 모두 드라이버 노드에 리턴하고 시각화 라이브러리 함수를 사용해 히스토그램을 만듬.
- 데이터를 샘플링해 드라이버 노드에 리턴, 리턴된 데이터를 이용해 시각화 한다.
(데이터셋의 행수가 너무 많으면 두번째 방법은 작업이 불가능하다)

In [13]:
# 데이터를 워커 노드에서 집계해서 워커 노드가 bin 리스트를 드라이버 노드에게 리턴하고 각 bin의개수를 드라이버 노드가 카운트
hist = fraud_df.select('balance').rdd.flatMap(lambda row: row).histogram(20)

In [82]:
len(hist[0])

21

In [91]:
!conda install -c anaconda matplotlib -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\a\Anaconda3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.5                |           py37_0         3.1 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.8.5-py37hc8dfbb8~ --> anaconda::conda-4.8.5-py37_0




conda-4.8.5          | 3.1 MB    |            |   0% 
conda-4.8.5          | 3.1 MB    |            |   1% 
conda-4.8.5          | 3.1 MB    | #8         |  18% 
conda-4.8.5          | 3.1 MB    | ###5       |  35% 
conda-4.8.5          | 3.1 MB    | #####3     |  53% 
conda-4.8.5          | 3.1 MB    | #######5   |  76% 
conda

In [14]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
data = {
    'bins': hist[0][:-1],
    'freq': hist[1]
}
fig = plt.figure(figsize=(12, 0))
ax = fig.add_subplot(1,1,1)
ax.bar(data['bins'], data['freq'], width=2000)
ax.set_title('Histogram of \'balance\'')
plt.show()

In [ ]:
from bokeh.io import output_notebook
output_notebook()
import holoviews as hv
hv.extension('bokeh')

In [ ]:
hv.Bars(data['freq'], 'freq', 'bins') + hv.Bars(data['bins'], 'freq', 'bins')

In [95]:
# 데이터의 양이 드라이버 노드에서 처리 할 수 있을 정도로 충분히 작다면,
# 데이터를 드라이버 노드로 가져와서 matplotlib의 hist() 함수 또는 Bokeh의 Histoghram() 함수를 사용해
# 히스토그램을 작성 할 수 있다.

data_driver = {'obs': fraud_df.select('balance').rdd.flatMap(lambda row: row).collect()}

In [96]:
len(data_driver['obs'])

10000000

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(1,1,1)
ax.hist(data_driver['obs'], bins=20)
ax.set_title('Histogram of \'balance\' using .hist()')
plt.show()

### Scatter 
변수간의 상호작용을 시각화 할 수 있다.
스파크에서는 어떠한 시각화 모듈도 제공하지 않는다.  
수십업개의 데이터를 동시에 시각화 하는것은 비현식적이다.  
데이터셋에서 0.02%로 셈플링하여 시각화를 해본다.

In [16]:
data_sample = fraud_df.sampleBy('gender', {0: 0.0002, 1: 0.0002}).select(numerical)

In [17]:
data_sample.printSchema()

root
 |-- balance: integer (nullable = true)
 |-- numTrans: integer (nullable = true)
 |-- numIntlTrans: integer (nullable = true)



In [ ]:
data_multi = dict([
    (elem, data_sample.select(elem).rdd.flatMap(lambda row:row).collect()) for elem in numerical
])

In [ ]:
hv.Scatter(data_multi, x='balance', y='numTrans')

In [19]:
spark.stop()